In [5]:
import pandas as pd
import numpy as np

# Load preprocessed data
trace_df = pd.read_csv("processed_trace_data.csv")
plaintexts = [p for p in trace_df['Plaintext']]
ciphertexts = [c for c in trace_df['Ciphertext']]
powers_at_t = trace_df['Power']

In [6]:
from des_helpers import dpa_select_function

# Constants
NUM_KEYS = 64  # Keys range from 0 to 63

dom_values = []

for i in range(NUM_KEYS):
    print(f"Computed DOM for key: {i}")
    sel_values = np.array([dpa_select_function(ct.upper(), i) for ct in ciphertexts])

    # Partition the power traces based on selection values
    group0_indices = [i for i, val in enumerate(sel_values) if val == 0]
    group1_indices = [i for i, val in enumerate(sel_values) if val == 1]
    
    # Extract power traces for each group
    group0_powers = [powers_at_t[i] for i in group0_indices]
    group1_powers = [powers_at_t[i] for i in group1_indices]
    
    # Compute average power consumption for each group at the time point of interest
    avg_group0 = np.mean(group0_powers)
    avg_group1 = np.mean(group1_powers)
    
    # Compute the difference of means (DOM)
    dom = avg_group1 - avg_group0 
    dom_values.append(dom)

Computed DOM for key: 0
Computed DOM for key: 1
Computed DOM for key: 2
Computed DOM for key: 3
Computed DOM for key: 4
Computed DOM for key: 5
Computed DOM for key: 6
Computed DOM for key: 7
Computed DOM for key: 8
Computed DOM for key: 9
Computed DOM for key: 10
Computed DOM for key: 11
Computed DOM for key: 12
Computed DOM for key: 13
Computed DOM for key: 14
Computed DOM for key: 15
Computed DOM for key: 16
Computed DOM for key: 17
Computed DOM for key: 18
Computed DOM for key: 19
Computed DOM for key: 20
Computed DOM for key: 21
Computed DOM for key: 22
Computed DOM for key: 23
Computed DOM for key: 24
Computed DOM for key: 25
Computed DOM for key: 26
Computed DOM for key: 27
Computed DOM for key: 28
Computed DOM for key: 29
Computed DOM for key: 30
Computed DOM for key: 31
Computed DOM for key: 32
Computed DOM for key: 33
Computed DOM for key: 34
Computed DOM for key: 35
Computed DOM for key: 36
Computed DOM for key: 37
Computed DOM for key: 38
Computed DOM for key: 39
Computed D

In [7]:
max_dom = max(dom_values)
correct_key_guess = dom_values.index(max_dom)
print(f"The key guess with the highest DOM is {correct_key_guess} with a value of {max_dom}")
print(f"The correct key is {60} with a value of {dom_values[60]}")

The key guess with the highest DOM is 15 with a value of 0.00012187199620955491
The correct key is 60 with a value of 7.318311279938539e-05


In [8]:
dom_values

[2.9431298920426507e-05,
 4.389556703983388e-05,
 2.8027042746570086e-05,
 1.261324681223197e-05,
 -6.622742869817622e-05,
 -2.0531007537641366e-05,
 3.4366833049845336e-05,
 2.3178004660978635e-06,
 8.487824737543614e-06,
 6.036434160833215e-06,
 -6.259062621792599e-05,
 1.1342119717552834e-05,
 8.874251624657603e-07,
 -7.41137119667501e-05,
 -7.58135749017591e-05,
 0.00012187199620955491,
 -5.9381395271479454e-05,
 -5.793761095819472e-05,
 -2.177268684805389e-05,
 3.416606566701563e-05,
 2.430753278215292e-05,
 -0.00012379469841251878,
 -0.00011642198490825761,
 -3.840725538943601e-05,
 5.8768116169852225e-05,
 -2.670196337112296e-05,
 5.13084031626837e-05,
 9.238820224398425e-05,
 0.00010734741613337151,
 6.639025189381087e-06,
 5.927873543602746e-05,
 1.0210964395004907e-05,
 0.00010970262948271914,
 0.00010370201618546682,
 6.989483858524381e-05,
 -2.9050039554834317e-05,
 8.299751602472427e-05,
 2.5501055633661562e-05,
 6.285237930758747e-05,
 7.297057234391391e-05,
 -6.013191291